In [1]:
import os
os.chdir("../")
%pwd

'd:\\python-projects\\chest-cancer-classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    base_model_updated: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [3]:
from src.cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])



    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.pth",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            base_model_updated="artifacts/prepare_base_model/base_model_updated.pth",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [5]:
import torch
from pytorch_lightning import Trainer
from src.cnnClassifier.components.prepare_base_model import MyImageClassifier
from src.cnnClassifier.components.model_trainer import TrainingDataModule

d:\python-projects\chest-cancer-classification\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.datamodule = None

    def load_model(self):
        checkpoint = torch.load(self.config.path_of_model, map_location="cpu", weights_only=False)
        state_dict = checkpoint['state_dict']
        base_model = torch.load(self.config.base_model_updated, map_location="cpu", weights_only=False)
        model = MyImageClassifier(model=base_model, config=self.config)
        model.load_state_dict(state_dict)
        self.model = model


    def load_data(self):
        self.datamodule = TrainingDataModule(self.config)
        self.datamodule.setup(stage="validate")  # or test

    def evaluation(self):
        self.load_model()
        self.load_data()

        trainer = Trainer(accelerator="cpu", logger=False)
        result = trainer.validate(self.model, datamodule=self.datamodule, verbose=True)
        self.score = result[0]
        
        return result
    
    
    def save_score(self):
        scores = {"loss": self.score.get("val_loss"),"accuracy": self.score.get("val_accuracy")}
        save_json(path=Path("scores.json"), data=scores)

In [7]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
   raise e

[2025-06-12 11:51:11,160] [14] [common] - INFO - YAML file loaded successfully: config\config.yaml
[2025-06-12 11:51:11,166] [14] [common] - INFO - YAML file loaded successfully: params.yaml
[2025-06-12 11:51:11,170] [27] [common] - INFO - Created directory at: artifacts
[2025-06-12 11:51:13,778] [108] [callback_connector] - INFO - Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
[2025-06-12 11:51:13,932] [156] [setup] - INFO - GPU available: True (cuda), used: False
[2025-06-12 11:51:13,937] [159] [setup] - INFO - TPU available: False, using: 0 TPU cores
[2025-06-12 11:51:13,938] [169] [setup] - INFO - HPU available: False, using: 0 HPUs


d:\python-projects\chest-cancer-classification\venv\Lib\site-packages\pytorch_lightning\trainer\setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


Validation DataLoader 0:  20%|██        | 1/5 [00:03<00:12,  0.31it/s]

d:\python-projects\chest-cancer-classification\venv\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('val_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
d:\python-projects\chest-cancer-classification\venv\Lib\site-packages\pytorch_lightning\core\module.py:512: You called `self.log('val_accuracy', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Validation DataLoader 0: 100%|██████████| 5/5 [00:12<00:00,  0.40it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_accuracy        │    0.8823529481887817     │
│         val_loss          │    0.4309086501598358     │
└───────────────────────────┴───────────────────────────┘

[2025-06-12 11:51:42,150] [41] [common] - INFO - json file saved at: scores.json


# Integrated with MLflow and Tensorboard

In [8]:
from src.cnnClassifier.entity.config_entity import LogConfig
from src.cnnClassifier.config.configuration import ConfigurationManager
from pytorch_lightning.loggers import MLFlowLogger, TensorBoardLogger

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig, logs:LogConfig):
        self.config = config
        self.logs = logs
        self.model = None
        self.datamodule = None
        self.tb_logger = TensorBoardLogger(str(self.logs.tensorboard / "evaluation"))
        self.mlflow_logger = MLFlowLogger(
            experiment_name="ChestCancerClassification_Evaluation",
            tracking_uri=str(self.logs.mlflow)
        )

    def load_model(self):
        checkpoint = torch.load(self.config.path_of_model, map_location="cpu", weights_only=False)
        state_dict = checkpoint['state_dict']
        base_model = torch.load(self.config.base_model_updated, map_location="cpu", weights_only=False)
        model = MyImageClassifier(model=base_model, config=self.config)
        model.load_state_dict(state_dict)
        self.model = model


    def load_data(self):
        self.datamodule = TrainingDataModule(self.config)
        self.datamodule.setup(stage="validate")  # or test

    def evaluation(self):
        self.load_model()
        self.load_data()

        trainer = Trainer(accelerator="cpu", logger=[self.tb_logger, self.mlflow_logger])
        result = trainer.validate(self.model, datamodule=self.datamodule, verbose=True)
        self.score = result[0]
        
        return result
    
    
    def save_score(self):
        scores = {"loss": self.score.get("val_loss"),"accuracy": self.score.get("val_accuracy")}
        save_json(path=Path("scores.json"), data=scores)

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    logs_config = config.get_log_config()
    evaluation = Evaluation(eval_config, logs_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
   raise e

[2025-06-12 11:51:42,276] [14] [common] - INFO - YAML file loaded successfully: config\config.yaml
[2025-06-12 11:51:42,287] [14] [common] - INFO - YAML file loaded successfully: params.yaml
[2025-06-12 11:51:42,293] [27] [common] - INFO - Created directory at: artifacts
[2025-06-12 11:51:42,302] [27] [common] - INFO - Created directory at: artifacts\logs
[2025-06-12 11:51:45,387] [108] [callback_connector] - INFO - Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
[2025-06-12 11:51:45,438] [156] [setup] - INFO - GPU available: True (cuda), used: False
[2025-06-12 11:51:45,440] [159] [setup] - INFO - TPU available: False, using: 0 TPU cores
[2025-06-12 11:51:45,441] [169] [setup] - INFO - HPU available: False, using: 0 HPUs
Validation DataLoader 0: 100%|██████████| 5/5 [00:11<00:00,  0.43it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_accuracy        │    0.8823529481887817     │
│         val_loss          │    0.4309086501598358     │
└───────────────────────────┴───────────────────────────┘

[2025-06-12 11:52:10,586] [41] [common] - INFO - json file saved at: scores.json
